In [1]:
 # Resources
# https://github.com/wanifarooq/NCOD/blob/main/README.md
# https://github.com/wanifarooq/NCOD/blob/main/NCOD.py
# https://github.com/wanifarooq/NCOD/blob/main/implementation/trainer/trainer.py
# https://arxiv.org/abs/2303.09470

In [2]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 35.2 MB/s eta 0:00:00


In [3]:
# !git clone --branch baselineCe https://github.com/Graph-Classification-Noisy-Label/hackaton.git
!git clone --branch baseline https://github.com/technahuynh/purple-hippo-project.git

Cloning into 'purple-hippo-project'...
remote: Enumerating objects: 178, done.
remote: Total 178 (delta 0), reused 0 (delta 0), pack-reused 178 (from 1)
Receiving objects: 100% (178/178), 253.39 MiB | 40.54 MiB/s, done.
Resolving deltas: 100% (54/54), done.


In [4]:
%cd /kaggle/working/purple-hippo-project
# %cd /kaggle/working/hackaton
# !rm -r /submission/*
!ls

/kaggle/working/purple-hippo-project
checkpoints  logs  main.py  README.md  requirements.txt  src  submission


In [5]:
!gdown --folder https://drive.google.com/drive/folders/1XHCHtrB3hDeqDfY6tDmfB9WVEOp8QIUd -O datasets

Retrieving folder contents
Retrieving folder 17o2SXJ5CIHZJ7760bOv5aiT5KQuyUseh A
Processing file 1_dueXK_W0HuMFCKlC8U_mdDR96_5WpTZ test.json.gz
Processing file 10FvydaYod4NwemgB2xfYqA6ewdlXcfPp train.json.gz
Retrieving folder 1RS4gD4tlZC4GmkRRCLXsmEKn14uANjn2 B
Processing file 1ehkTEvDVo5yti0yuXWNosSZRajouScMo test.json.gz
Processing file 1TzwivNX5NdMH5QFBB3SJ-k0iJukXC7O_ train.json.gz
Retrieving folder 1IKovc5_Jr3dThMtIwqG68vL9WHj2Bs3U C
Processing file 1Yfix4FoeHqJd3DpoYHk5do36Gfm7-IuH test.json.gz
Processing file 1gXg3jRTkAye1A18dh4ju7nMaxMOfZbns train.json.gz
Retrieving folder 13AXnY9nQLGpnXMaGJ3Jyfy0WRqkRiVZ8 D
Processing file 1B4YMJQAHoU4rgWtywVQZWAntCepEWX25 test.json.gz
Processing file 1ce7J2NJc-80_LBmzDAEtGM0K9G-vLvAC train.json.gz
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From (original): https://drive.google.com/uc?id=1_dueXK_W0HuMFCKlC8U_mdDR96_5WpTZ
From (redirected): https://drive.google.com/uc?

In [6]:
!ls -lh datasets

total 16K
drwxr-xr-x 2 root root 4.0K May 30 11:47 A
drwxr-xr-x 2 root root 4.0K May 30 11:47 B
drwxr-xr-x 2 root root 4.0K May 30 11:48 C
drwxr-xr-x 2 root root 4.0K May 30 11:48 D


In [7]:
import os
import torch
import pandas as pd
import matplotlib.pyplot as plt
import logging
from tqdm import tqdm
from torch_geometric.loader import DataLoader
from torch.utils.data import random_split
# Load utility functions from cloned repository
from src.loadData import GraphDataset
from src.utils import set_seed
from src.models import GNN
import argparse
# NCOD loss
import torch.nn.functional as F
import torch.nn as nn
import math
import numpy as np

# Set the random seed
set_seed()


In [8]:
def add_zeros(data):
    data.x = torch.zeros(data.num_nodes, dtype=torch.long)
    return data

In [9]:
def train(data_loader, model, optimizer, criterion, device, save_checkpoints, checkpoint_path, current_epoch):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    for data in tqdm(data_loader, desc="Iterating training graphs", unit="batch"):
        data = data.to(device)
        optimizer.zero_grad()
        output,_ = model(data)
        loss = criterion(output, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        pred = output.argmax(dim=1)
        correct += (pred == data.y).sum().item()
        total += data.y.size(0)

    # Save checkpoints if required
    if save_checkpoints:
        checkpoint_file = f"{checkpoint_path}_epoch_{current_epoch + 1}_ncod.pth"
        torch.save(model.state_dict(), checkpoint_file)
        print(f"Checkpoint saved at {checkpoint_file}")

    return total_loss / len(data_loader),  correct / total

In [10]:
def evaluate(data_loader, model, device, calculate_accuracy=False):
    model.eval()
    correct = 0
    total = 0
    predictions = []
    total_loss = 0
    criterion = torch.nn.CrossEntropyLoss()
    with torch.no_grad():
        for data in tqdm(data_loader, desc="Iterating eval graphs", unit="batch"):
            data = data.to(device)
            output,_ = model(data)
            pred = output.argmax(dim=1)
            
            if calculate_accuracy:
                correct += (pred == data.y).sum().item()
                total += data.y.size(0)
                total_loss += criterion(output, data.y).item()
            else:
                predictions.extend(pred.cpu().numpy())
    if calculate_accuracy:
        accuracy = correct / total
        return  total_loss / len(data_loader),accuracy
    return predictions

In [11]:
def save_predictions(predictions, test_path):
    script_dir = os.getcwd() 
    submission_folder = os.path.join(script_dir, "submission")
    test_dir_name = os.path.basename(os.path.dirname(test_path))
    
    os.makedirs(submission_folder, exist_ok=True)
    
    output_csv_path = os.path.join(submission_folder, f"testset_{test_dir_name}.csv")
    
    test_graph_ids = list(range(len(predictions)))
    output_df = pd.DataFrame({
        "id": test_graph_ids,
        "pred": predictions
    })
    
    output_df.to_csv(output_csv_path, index=False)
    print(f"Predictions saved to {output_csv_path}")

In [12]:
def plot_training_progress(train_losses, train_accuracies, output_dir):
    epochs = range(1, len(train_losses) + 1)
    plt.figure(figsize=(12, 6))

    # Plot loss
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_losses, label="Training Loss", color='blue')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Loss per Epoch')

    # Plot accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_accuracies, label="Training Accuracy", color='green')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Training Accuracy per Epoch')

    # Save plots in the current directory
    os.makedirs(output_dir, exist_ok=True)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, "training_progress.png"))
    plt.close()

In [13]:
# Basic setup for early stopping criteria
patience = 50  # epochs to wait after no improvement
delta = 0.01  # minimum change in the monitored metric
best_val_loss = float("inf")  # best validation loss to compare against
no_improvement_count = 0  # count of epochs with no improvement

class EarlyStopping:
    def __init__(self, patience=5, delta=0, verbose=False):
        self.patience = patience
        self.delta = delta
        self.verbose = verbose
        self.best_loss = None
        self.no_improvement_count = 0
        self.stop_training = False
    
    def check_early_stop(self, val_loss):
        if self.best_loss is None or val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            self.no_improvement_count = 0
        else:
            self.no_improvement_count += 1
            if self.no_improvement_count >= self.patience:
                self.stop_training = True
                if self.verbose:
                    print("Stopping early as no improvement has been observed.")
# Initialize early stopping
early_stopping = EarlyStopping(patience=patience, delta=delta, verbose=True)

In [14]:
def get_user_input(prompt, default=None, required=False, type_cast=str):
    while True:
        user_input = input(f"{prompt} [{default}]: ")
        if user_input == "" and required:
            print("This field is required. Please enter a value.")
            continue
        if user_input == "" and default is not None:
            return default
        if user_input == "" and not required:
            return None
        try:
            return type_cast(user_input)
        except ValueError:
            print(f"Invalid input. Please enter a valid {type_cast.__name__}.")

def get_arguments():
    args = {}
    args['train_path'] = '/kaggle/working/purple-hippo-project/datasets/D/train.json.gz'
    args['test_path'] = '/kaggle/working/purple-hippo-project/datasets/D/test.json.gz'
    # args['train_path'] = '/kaggle/working/hackaton/datasets/A/train.json.gz'
    # args['test_path'] = '/kaggle/working/hackaton/datasets/A/test.json.gz'
    args['num_checkpoints'] = int(5)
    args['device'] = int(1)
    args['gnn'] = 'gin-virtual'
    args['drop_ratio'] = float(0.1)
    args['num_layer'] = int(5)
    args['emb_dim'] = int(300)
    args['batch_size'] = int(32)
    args['epochs'] = int(200)
    args['baseline_mode'] = int(3)
    args['noise_prob'] = float(0.2)
    
    return argparse.Namespace(**args)
def populate_args(args):
    print("Arguments received:")
    for key, value in vars(args).items():
        print(f"{key}: {value}")


In [15]:
def populate_args(args):
    print("Arguments received:")
    for key, value in vars(args).items():
        print(f"{key}: {value}")
args = get_arguments()
populate_args(args)

Arguments received:
train_path: /kaggle/working/purple-hippo-project/datasets/D/train.json.gz
test_path: /kaggle/working/purple-hippo-project/datasets/D/test.json.gz
num_checkpoints: 5
device: 1
gnn: gin-virtual
drop_ratio: 0.1
num_layer: 5
emb_dim: 300
batch_size: 32
epochs: 200
baseline_mode: 3
noise_prob: 0.2


In [16]:
class NoisyCrossEntropyLoss(torch.nn.Module):
    def __init__(self, p_noisy):
        super().__init__()
        self.p = p_noisy
        self.ce = torch.nn.CrossEntropyLoss(reduction='none')

    def forward(self, logits, targets):
        losses = self.ce(logits, targets)
        weights = (1 - self.p) + self.p * (1 - torch.nn.functional.one_hot(targets, num_classes=logits.size(1)).float().sum(dim=1))
        return (losses * weights).mean()

In [17]:
cross_entropy_val = nn.CrossEntropyLoss
mean = 1e-8
std  = 1e-9

class ncodLoss(nn.Module):
    def __init__(self, labels, n, C, ratio_consistency=0, ratio_balance=0):
        super(ncodLoss, self).__init__()

        self.C = C
        self.USE_CUDA = torch.cuda.is_available()
        self.n = n
        self.ratio_consistency = ratio_consistency
        self.ratio_balance = ratio_balance
        self.u = nn.Parameter(torch.empty(n, 1, dtype=torch.float32))
        self.init_param(mean=mean,std=std)
        self.beginning = True
        self.prev_phi_x_i = torch.rand((n, encoder_features))
        self.phi_c = torch.rand((C, encoder_features))
        # self.labels = labels
        self.labels = labels.cpu().numpy() if isinstance(labels, torch.Tensor) else labels
        self.bins = []

        # for i in range(0, C):
        #     self.bins.append(np.where(self.labels == i)[0])
        # Create bins for each class
        self.bins = []
        for i in range(C):
            self.bins.append(np.where(self.labels == i)[0])
        # Initialize h_c_bar_T
        self.h_c_bar_T = None

    def init_param(self, mean= 1e-8, std= 1e-9):
        torch.nn.init.normal_(self.u, mean=mean, std=std)
        
    def forward(self, index, f_x_i, y, phi_x_i, flag, epoch):
        # Move tensors to same device
        device = f_x_i.device
        if self.prev_phi_x_i.device != device:
            self.prev_phi_x_i = self.prev_phi_x_i.to(device)
        if self.phi_c.device != device:
            self.phi_c = self.phi_c.to(device)
            
        if len(f_x_i) > len(index):
            f_x_i_1, f_x_i_2 = torch.chunk(f_x_i, 2)
            phi_x_i_1, phi_x_i_2 = torch.chunk(phi_x_i, 2)
        else:
            f_x_i_1 = f_x_i
            phi_x_i_1 = phi_x_i
            f_x_i_2 = None
            phi_x_i_2 = None
            
        f_x_i_1 = f_x_i
        phi_x_i_1 = phi_x_i
        # TODO
        eps = 1e-4
        # Get corresponding u values
        u = self.u[index]
        
        # Update class prototypes at the beginning of each epoch
        if flag == 0:  # Beginning of epoch
            if self.beginning or epoch == 0:
                percent = max(1, math.ceil((50 - (50 / total_epochs) * epoch) + 50))
                
                for i in range(len(self.bins)):
                    if len(self.bins[i]) > 0:  # Check if class has samples
                        class_u = self.u.detach()[self.bins[i]]
                        bottomK = max(1, int((len(class_u) / 100) * percent))
                        
                        if len(class_u) >= bottomK:
                            important_indexs = torch.topk(class_u.squeeze(), bottomK, largest=False, dim=0)[1]
                            selected_features = self.prev_phi_x_i[self.bins[i]][important_indexs]
                            self.phi_c[i] = torch.mean(selected_features, dim=0)
                
                # Normalize class prototypes
                phi_c_norm = self.phi_c.norm(p=2, dim=1, keepdim=True)
                phi_c_norm = torch.clamp(phi_c_norm, min=eps)  # Avoid division by zero
                h_c_bar = self.phi_c.div(phi_c_norm)
                self.h_c_bar_T = torch.transpose(h_c_bar, 0, 1)
                
            self.beginning = False
        
        # Update previous features
        self.prev_phi_x_i[index] = phi_x_i_1.detach()
        
        # Compute softmax predictions
        f_x_softmax = F.softmax(f_x_i_1, dim=1)
        
        # Compute normalized features
        phi_x_i_1_norm = phi_x_i_1.detach().norm(p=2, dim=1, keepdim=True)
        phi_x_i_1_norm = torch.clamp(phi_x_i_1_norm, min=eps)
        h_i = phi_x_i_1.detach().div(phi_x_i_1_norm)
        
        # Compute similarity with class prototypes
        if self.h_c_bar_T is not None:
            y_bar = torch.mm(h_i, self.h_c_bar_T)
            y_bar = y_bar * y.float()
            y_bar_max = (y_bar > 0.0).float()
            y_bar = y_bar * y_bar_max
        else:
            y_bar = y.float()
        
        # Apply learnable noise
        u_expanded = u * y.float()
        f_x_softmax = torch.clamp((f_x_softmax + u_expanded.detach()), min=eps, max=1.0)
        
        # Compute main loss
        L1 = torch.mean(-torch.sum(y_bar * torch.log(f_x_softmax), dim=1))
        
        # Compute consistency loss for u
        y_hat = self.soft_to_hard(f_x_i_1.detach())
        L2 = F.mse_loss((y_hat + u_expanded), y.float(), reduction='mean')
        L1 += L2
        
        # Balance loss
        if self.ratio_balance > 0:
            avg_prediction = torch.mean(f_x_softmax, dim=0)
            prior_distr = (1.0 / self.C) * torch.ones_like(avg_prediction)
            avg_prediction = torch.clamp(avg_prediction, min=eps, max=1.0)
            balance_kl = torch.mean(-(prior_distr * torch.log(avg_prediction)).sum())
            L1 += self.ratio_balance * balance_kl
        
        # Consistency loss for augmented data
        if f_x_i_2 is not None and self.ratio_consistency > 0:
            consistency_loss = self.consistency_loss(f_x_i_1, f_x_i_2)
            L1 += self.ratio_consistency * torch.mean(consistency_loss)
        
        return L1
    
    def consistency_loss(self, f_x_i_1, f_x_i_2):
        preds1 = F.softmax(f_x_i_1, dim=1).detach()
        preds2 = F.log_softmax(f_x_i_2, dim=1)
        loss_kldiv = F.kl_div(preds2, preds1, reduction='none')
        loss_kldiv = torch.sum(loss_kldiv, dim=1)
        return loss_kldiv
    
    def soft_to_hard(self, x):
        with torch.no_grad():
            device = x.device
            hard_labels = torch.zeros(len(x), self.C, device=device)
            hard_labels.scatter_(1, x.argmax(dim=1).view(-1, 1), 1)
            return hard_labels


In [18]:
script_dir = os.getcwd() 
# device = torch.device(f"cuda:{args.device}" if torch.cuda.is_available() else "cpu")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_checkpoints = args.num_checkpoints if args.num_checkpoints else 3
    
if args.gnn == 'gin':
    model = GNN(gnn_type='gin', num_class=6, num_layer=args.num_layer, emb_dim=args.emb_dim, drop_ratio=args.drop_ratio, virtual_node=False).to(device)
elif args.gnn == 'gin-virtual':
    model = GNN(gnn_type='gin', num_class=6, num_layer=args.num_layer, emb_dim=args.emb_dim, drop_ratio=args.drop_ratio, virtual_node=True).to(device)
elif args.gnn == 'gcn':
    model = GNN(gnn_type='gcn', num_class=6, num_layer=args.num_layer, emb_dim=args.emb_dim, drop_ratio=args.drop_ratio, virtual_node=False).to(device)
elif args.gnn == 'gcn-virtual':
    model = GNN(gnn_type='gcn', num_class=6, num_layer=args.num_layer, emb_dim=args.emb_dim, drop_ratio=args.drop_ratio, virtual_node=True).to(device)
else:
    raise ValueError('Invalid GNN type')
    
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# criterion = torch.nn.CrossEntropyLoss()
if args.baseline_mode == 2:
    criterion = NoisyCrossEntropyLoss(args.noise_prob)
else:
    criterion = torch.nn.CrossEntropyLoss()
# elif args.baseline_mode == 3:
#     criterion = ncodLoss(labels, samples, classes)

# NCOD Loss
encoder_features = args.emb_dim #512
total_epochs = args.epochs
num_classes = 6
# learnable param u of size n=no.samples

In [19]:
test_dir_name = os.path.basename(os.path.dirname(args.test_path))
logs_folder = os.path.join(script_dir, "logs", test_dir_name)
log_file = os.path.join(logs_folder, "training.log")
os.makedirs(os.path.dirname(log_file), exist_ok=True)
logging.basicConfig(filename=log_file, level=logging.INFO, format='%(asctime)s - %(message)s')
logging.getLogger().addHandler(logging.StreamHandler())

checkpoint_path = os.path.join(script_dir, "checkpoints", f"model_{test_dir_name}_best.pth")
checkpoints_folder = os.path.join(script_dir, "checkpoints", test_dir_name)
os.makedirs(checkpoints_folder, exist_ok=True)

In [20]:
if os.path.exists(checkpoint_path) and not args.train_path:
    model.load_state_dict(torch.load(checkpoint_path))
    print(f"Loaded best model from {checkpoint_path}")

In [21]:
# Load data
full_dataset = GraphDataset(args.train_path, transform=add_zeros)
val_size = int(0.2 * len(full_dataset))
train_size = len(full_dataset) - val_size
generator = torch.Generator().manual_seed(12)
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size], generator=generator)
train_loader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=args.batch_size, shuffle=False)

# Train loop var
num_epochs = args.epochs
best_val_accuracy = 0.0   
train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []

In [22]:
# Get labels for NCOD loss
labels = []
train_indices = []
idx_counter = 0
# Collect all training labels and create index mapping
for batch_idx, data in enumerate(train_loader):
    batch_labels = data.y.cpu().numpy()
    labels.extend(batch_labels)
    batch_indices = list(range(idx_counter, idx_counter + len(batch_labels)))
    train_indices.extend(batch_indices)
    idx_counter += len(batch_labels)
labels = np.array(labels)
train_size = len(labels)

In [23]:
# # Load criterion
# labels = torch.stack([data.y for data in train_dataset if data.y is not None])
# if args.baseline_mode == 3:
#     criterion = ncodLoss(labels, train_size, num_classes)
#     u = nn.Parameter(torch.empty(train_size, 1, dtype=torch.float32))
#     optimizer_u = torch.optim.SGD([u], lr=0.3, weight_decay=0)
#     # TODO turn off self.beginning??
# Initialize NCOD loss
if args.baseline_mode == 3:
    criterion = ncodLoss(labels, train_size, num_classes)
    criterion = criterion.to(device)
    
    # Setup optimizers
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    optimizer_u = torch.optim.SGD(criterion.parameters(), lr=0.3, weight_decay=0)
else:
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    optimizer_u = None

In [24]:
if num_checkpoints > 1:
    checkpoint_intervals = [int((i + 1) * num_epochs / num_checkpoints) for i in range(num_checkpoints)]
else:
    checkpoint_intervals = [num_epochs]
# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    idx = 0
    with tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch") as progress:
        for batch_idx, data in enumerate(progress):
            data = data.to(device)
            label = data.y.long().to(device)
            batch_size = data.y.size(0)
            
            # Get model outputs
            f_x_i, phi_x_i = model(data)
            
            if args.baseline_mode == 3:
                # NCOD loss
                y_one_hot = F.one_hot(label, num_classes).float()
                current_indices = torch.arange(idx, idx + batch_size, device=device)
                
                loss = criterion(current_indices, f_x_i, y_one_hot, phi_x_i, batch_idx, epoch)
                
                # Optimization step
                optimizer.zero_grad()
                if optimizer_u is not None:
                    optimizer_u.zero_grad()
                
                loss.backward()
                optimizer.step()
                if optimizer_u is not None:
                    optimizer_u.step()
                    
            else:
                # Standard cross-entropy loss
                loss = criterion(f_x_i, label)
                
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            # Statistics
            total_loss += loss.item()
            pred = f_x_i.argmax(dim=1)
            correct += (pred == label).sum().item()
            total += label.size(0)
            idx += batch_size

    # Calculate epoch metrics
    train_loss = total_loss / len(train_loader)
    train_acc = correct / total
    
    # Validation
    val_loss, val_acc = evaluate(val_loader, model, device, calculate_accuracy=True)
    
    # Logging
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")
    if hasattr(logging, 'info'):
        logging.info(f"Epoch {epoch + 1}/{num_epochs}, Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")
    
    # Store metrics
    train_losses.append(train_loss)
    train_accuracies.append(train_acc)
    val_losses.append(val_loss)
    val_accuracies.append(val_acc)
    
    # Save best model
    if val_acc > best_val_accuracy:
        best_val_accuracy = val_acc
        torch.save(model.state_dict(), checkpoint_path)
        print(f"Best model updated and saved at {checkpoint_path}")
    
    # Save checkpoints
    if (epoch + 1) in checkpoint_intervals:
        checkpoint_file = f"{checkpoint_path}_epoch_{epoch + 1}.pth"
        torch.save(model.state_dict(), checkpoint_file)
        print(f"Checkpoint saved at {checkpoint_file}")
    # Reset beginning flag for NCOD
    if args.baseline_mode == 3:
        criterion.beginning = True
        
    # Early stopping
    early_stopping.check_early_stop(val_loss)
    if early_stopping.stop_training:
        print(f"Early stopping at epoch {epoch}")
        break

Iterating eval graphs: 100%|██████████| 65/65 [00:09<00:00,  6.79batch/s]
Epoch 1/200, Loss: 0.2662, Train Acc: 0.2267, Val Acc: 0.1124


Epoch 1/200, Loss: 0.2662, Train Acc: 0.2267, Val Acc: 0.1124
Best model updated and saved at /kaggle/working/purple-hippo-project/checkpoints/model_D_best.pth


Iterating eval graphs: 100%|██████████| 65/65 [00:09<00:00,  6.84batch/s]
Epoch 2/200, Loss: 1.0122, Train Acc: 0.2952, Val Acc: 0.3064


Epoch 2/200, Loss: 1.0122, Train Acc: 0.2952, Val Acc: 0.3064
Best model updated and saved at /kaggle/working/purple-hippo-project/checkpoints/model_D_best.pth


Iterating eval graphs: 100%|██████████| 65/65 [00:09<00:00,  6.88batch/s]
Epoch 3/200, Loss: 0.4872, Train Acc: 0.2881, Val Acc: 0.2889


...

Epoch 126/200, Loss: 0.1963, Train Acc: 0.8242, Val Acc: 0.7223


Iterating eval graphs: 100%|██████████| 65/65 [00:09<00:00,  6.82batch/s]
Epoch 127/200, Loss: 0.1933, Train Acc: 0.8276, Val Acc: 0.7335


Epoch 127/200, Loss: 0.1933, Train Acc: 0.8276, Val Acc: 0.7335


Iterating eval graphs: 100%|██████████| 65/65 [00:09<00:00,  6.89batch/s]
Epoch 128/200, Loss: 0.2027, Train Acc: 0.8253, Val Acc: 0.6581


Epoch 128/200, Loss: 0.2027, Train Acc: 0.8253, Val Acc: 0.6581
Stopping early as no improvement has been observed.
Early stopping at epoch 127


In [25]:
plot_training_progress(train_losses, train_accuracies, os.path.join(logs_folder, "plots"))
plot_training_progress(val_losses, val_accuracies, os.path.join(logs_folder, "plotsVal"))

In [26]:
import gc
del train_dataset
del train_loader
del full_dataset
del val_dataset
del val_loader
gc.collect()

13518

In [27]:
test_dataset = GraphDataset(args.test_path, transform=add_zeros)
test_loader = DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False)
    

In [28]:
model.load_state_dict(torch.load(checkpoint_path))
predictions = evaluate(test_loader, model, device, calculate_accuracy=False)
save_predictions(predictions, args.test_path)

<ipython-input-28-10583d8b995c>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(checkpoint_path))
Iterating eval graphs: 100%|██████████| 7

Predictions saved to /kaggle/working/purple-hippo-project/submission/testset_D.csv
